In [7]:


from datasets import load_from_disk

/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import os
from sklearn.preprocessing import StandardScaler

from astrodino import format_with_env
from property_utils.models import few_shot, zero_shot

ModuleNotFoundError: No module named 'pyro'

In [4]:
ASTROCLIP_ROOT = format_with_env("{ASTROCLIP_ROOT}")
QUASARS_ROOT = f"{ASTROCLIP_ROOT}/data/sample_0.1/"

In [5]:
image_models = ["astroclip_image", "astrodino"]
spectrum_models = ["astroclip_spectrum", "specformer"]

In [6]:
train_path = os.path.join(QUASARS_ROOT, "train_dataset")
test_path = os.path.join(QUASARS_ROOT, "test_dataset")

In [ ]:
train_dataset=load_from_disk(train_path)
test_dataset=load_from_disk(test_path)

In [ ]:
z_train = train_dataset["z"]
z_test = test_dataset["z"]

In [ ]:
scaler = {"mean": z_train.mean(), "std": z_train.std()}
z_train = (z_train - scaler["mean"]) / scaler["std"]

In [ ]:
print(
    "Size of training set:",
    len(train_dataset),
    "\nSize of test set:",
    len(test_dataset),
)

# Redshift estimation from image embeddings

In [ ]:
# Get data
data = {}
for model in image_models:
    data[model] = {}
    X_train, X_test = (
        train_dataset[model + "_embeddings"],
        test_dataset[model + "_embeddings"],
    )
    embedding_scaler = StandardScaler().fit(X_train)
    data[model]["train"] = embedding_scaler.transform(X_train)
    data[model]["test"] = embedding_scaler.transform(X_test)

In [ ]:
# Perfrom knn and mlp
preds_knn, preds_mlp = {}, {}
for key in data.keys():
    print(f"Evaluating {key} model...")
    raw_preds_knn = zero_shot(data[key]["train"], z_train, data[key]["test"])
    raw_preds_mlp = few_shot(
        model, data[key]["train"], z_train, data[key]["test"], hidden_dims=[32]
    ).squeeze()
    preds_knn[key] = raw_preds_knn * scaler["std"] + scaler["mean"]
    preds_mlp[key] = raw_preds_mlp * scaler["std"] + scaler["mean"]

In [ ]:
preds_knn["AstroCLIP"] = preds_knn.pop("astroclip_image")
preds_mlp["AstroCLIP"] = preds_mlp.pop("astroclip_image")